In [0]:
#!pip install tf-nightly-gpu-2.0-preview
!pip3 install soundfile
!pip3 install python_speech_features


import urllib
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# download data: 
!cp -r "/content/drive/My Drive/WordList/mfcc_final" .

!cp -r "/content/drive/My Drive/WordList/labels_final" .

print("done download")

# drive_url = 'https://drive.google.com/file/d/1GXG9bH74UV0htJ2giYsstL3WNB-CYoJc/view?usp=sharing'
# file_name = 'speech.gz'
# urllib.request.urlretrieve(drive_url, file_name)
# print('Download completed!')
# print("done importing")






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
done download


#MFCC EXTRACTION

In [0]:

import python_speech_features as psf
import soundfile as sf
def psfmfcc(s,sr,nf=2048):
  
  L=psf.mfcc(s,sr,winlen=0.025,winstep=0.01,numcep=20,
    nfilt=26,nfft=2048,lowfreq=0,highfreq=None,preemph=0.97,
    ceplifter=22,appendEnergy=True)
  return(L)

#Checking data

In [0]:
###Loading data
import numpy as np
import pickle
import pandas
#load mfcc
file=open("mfcc_final",'rb')
data=pickle.load(file)
file.close()
#load labels
file=open("labels_final",'rb')
labels=pickle.load(file)
file.close()
lens=[len(recording) for recording in data]

data=np.asarray(data)
data.shape

avg_len=np.mean(lens)
max=np.max(lens)
print("maxlen=",max)
std=np.std(lens)
print("std=",std)
n_labels=len(np.unique(labels))

print("avg len=",avg_len)

print(data.shape)



print(data[1].shape)




len(labels)

labels[1]
labels [10000]

maxlen= 143
std= 15.060360756121268
avg len= 99.36507381793211
(163104,)
(99, 20)


'eight'

#Padding data
  

In [0]:
#padding our data:

#We pad our data to give it the same length in order to feed it to the DNN

import numpy as np

a=np.asarray(data)
X=[]

max_len=130

for mfcc in a:
  
  mfcc=mfcc[:max_len]
  mfcc=np.pad(mfcc,((0,max_len-len(mfcc)),(0,0)),mode='constant')
  X.append(mfcc)
  
X=np.asarray(X)
print("X shape is ",X.shape)

#making sure data is good:
from collections import Counter
print(Counter(labels))


X shape is  (163104, 130, 20)
Counter({'eight': 10194, 'five': 10194, 'four': 10194, 'nine': 10194, 'seven': 10194, 'six': 10194, 'three': 10194, 'two': 10194, 'zero': 10194, 'one': 10194, 'left': 10194, 'marvin': 10194, 'no': 10194, 'right': 10194, 'up': 10194, 'yes': 10194})


#Changing labels to numbers and splitting data

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

Y=labels

#giving our labels an id:

dic={}
unk=np.unique(Y)
for i in range(len(unk)):
  dic[unk[i]]=i
  
print(dic)
n_labels=[dic[i] for i in Y]


#creating training and testing sets:
    
X_train,X_test, Y_train,Y_test = train_test_split(X,n_labels,test_size=0.3)

k=dic
file=open("dict2",'wb')
pickle.dump(k,file,protocol=2)
file.close()


its me Counter({'eight': 10194, 'five': 10194, 'four': 10194, 'nine': 10194, 'seven': 10194, 'six': 10194, 'three': 10194, 'two': 10194, 'zero': 10194, 'one': 10194, 'left': 10194, 'marvin': 10194, 'no': 10194, 'right': 10194, 'up': 10194, 'yes': 10194})
{'eight': 0, 'five': 1, 'four': 2, 'left': 3, 'marvin': 4, 'nine': 5, 'no': 6, 'one': 7, 'right': 8, 'seven': 9, 'six': 10, 'three': 11, 'two': 12, 'up': 13, 'yes': 14, 'zero': 15}


#Running code on GPU because it's faster (and cooler)

In [0]:
device_name=tf.test.gpu_device_name()
if device_name!= '/device:GPU:0':
     raise SystemError("GPU not found")
print('Found GPU at : {}' .format(device_name)) #checking 

Found GPU at : /device:GPU:0


#Creating and running model (DNN)

In [0]:
import tensorflow as tf


#run code on gpu
with tf.device('/gpu:0'):
    model=tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(max_len, 13)),
        tf.keras.layers.Dropout(0.20)
        tf.keras.layers.Dense(350,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.18),
        tf.keras.layers.Dense(200,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.18),

        tf.keras.layers.Dense(130,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.001)),

        tf.keras.layers.Dropout(0.12),



        tf.keras.layers.Dense(31,activation=tf.nn.softmax,kernel_regularizer=tf.keras.regularizers.l2(0.001))
    ])


    #compile model :

    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # use the model:

   # Y_test = to_categorical(Y_test)
    #Y_train =to_categorical(Y_train)
    loss,accuracy=model.evaluate(X_test,Y_test)
    history=model.fit(X_train,Y_train,epochs=70,batch_size=90,validation_data=[X_test,Y_test])
    loss,accuracy=model.evaluate(X_test,Y_test)







ValueError: ignored

In [0]:
!tensorboard --logdir /tmp/retrain_logs

#Second architecture (CNN : Best so far)

In [0]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

with tf.device('/gpu:0'):

  X_train_3D = X_train.reshape(X_train.shape[0], max_len, 20, 1)
  X_test_3D = X_test.reshape(X_test.shape[0], max_len, 20, 1)
  print("shape X" ,X_train.shape)

  model = Sequential()
  model.add(Conv2D(60, kernel_size=(10,20), strides=(5,20), activation='relu', input_shape=(max_len, 20, 1)))
  model.add(MaxPooling2D(pool_size=(5, 1 )))
  model.add(Dropout(0.18))
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.12))
  model.add(Dense(300, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(Dropout(0.12))
  
  model.add(Dense(180, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(Dropout(0.12))
  model.add(Dense(100, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(Dropout(0.12))
  model.add(Dense(31, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])
  # simple early stopping
  es = EarlyStopping(monitor='val_acc', mode='min', verbose=0,patience=20)
  mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
  #train model
  model.fit(X_train_3D, Y_train, batch_size=80, epochs=100, verbose=1, validation_data=(X_test_3D, Y_test),callbacks=[es, mc])

shape X (114172, 130, 20)
Train on 114172 samples, validate on 48932 samples
Epoch 1/100
113840/114172 [============================>.] - ETA: 0s - loss: 1.2544 - acc: 0.7253
Epoch 00001: val_acc improved from -inf to 0.87707, saving model to best_model.h5
114172/114172 [==============================] - 11s 100us/sample - loss: 1.2528 - acc: 0.7256 - val_loss: 0.6277 - val_acc: 0.8771
Epoch 2/100
113760/114172 [============================>.] - ETA: 0s - loss: 0.6658 - acc: 0.8497
Epoch 00002: val_acc improved from 0.87707 to 0.89851, saving model to best_model.h5
114172/114172 [==============================] - 10s 90us/sample - loss: 0.6658 - acc: 0.8496 - val_loss: 0.4640 - val_acc: 0.8985
Epoch 3/100
113600/114172 [============================>.] - ETA: 0s - loss: 0.5491 - acc: 0.8707
Epoch 00003: val_acc improved from 0.89851 to 0.91476, saving model to best_model.h5
114172/114172 [==============================] - 10s 88us/sample - loss: 0.5494 - acc: 0.8706 - val_loss: 0.3919 -

#Recursive DNN:

In [0]:

import tensorflow as tf
#from tensorflow.keras.utils.np_utils import to_categorical




# tf.keras.layers.Dense(200,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
with tf.device('/gpu:0'):
    model=tf.keras.models.Sequential([
        
        #tf.keras.layers.CuDNNLSTM(275, input_shape=(max_len,13)),
        tf.keras.layers.LSTM(275, input_shape=(max_len,13)),
       
        tf.keras.layers.Dropout(0.30),
        
        tf.keras.layers.Dense(200,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.001)),

        tf.keras.layers.Dropout(0.30),
        tf.keras.layers.Dense(110,activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.001)),

        tf.keras.layers.Dropout(0.25),
        
        tf.keras.layers.Dense(31,activation=tf.nn.softmax,kernel_regularizer=tf.keras.regularizers.l2(0.001))
    ])


    #compile model :
    opt = tf.keras.optimizers.Adam(lr = 1e-3,decay = 1e-5)

    model.compile(optimizer=opt, 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # use the model:

    history=model.fit(X_train,Y_train,epochs=50,batch_size=100,validation_data=[X_test,Y_test])
    loss,accuracy=model.evaluate(X_test,Y_test)
    
    from tensorflow.keras.models import save_model

    # Creates a HDF5 file 'my_model.h5'


    save_model(model,'LSTM_NORMAL.h5',True,False)

#Save Model


In [0]:
from tensorflow.keras.models import save_model

# Creates a HDF5 file 'my_model.h5'


save_model(model,'LSTM2.h5',True,False)

#Load Model


In [0]:
mo = tf.keras.models.load_model('my_model_c.h5')
mo = tf.keras.models.load_model('my_model_c.h5')
mo.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']) 
loss,accuracy=mo.evaluate(X_train,Y_train)

In [0]:
#Generate confusion matrix:
import sklearn
Y=model.predict(X_test)
Y_predict=[np.argmax(Y[i]) for i in range(len(Y))]

sklearn.metrics.confusion_matrix(Y_test,Y_predict)



In [0]:
#plotting loss and accuracy over time
history_dict = history.history
print(history_dict.keys())
import matplotlib.pyplot as plt

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()